In [1]:
%pip install pandas matplotlib numpy


[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
csv_file = "./dataset/train.csv"
test_csv_file = "./dataset/test.csv"
train_df = pd.read_csv(csv_file)
test_df = pd.read_csv(test_csv_file)
print(train_df.shape)
print( train_df.columns)
print(train_df.info())
print(train_df.isnull().sum().sort_values(ascending=False))
# print(df.head())

(8693, 14)
Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-n

In [4]:
def onehot(df : pd.DataFrame,keyName : str) -> pd.DataFrame:
    for key in df[keyName].value_counts().index:
        df[key] = df[keyName] == key
        df[key] = df[key].astype(int)
    return df.drop(keyName, axis=1)
def mapValue(df : pd.DataFrame,keyName : str) -> pd.DataFrame:
    mapDict = {}
    for idx, key in enumerate(df[keyName].value_counts().index):
        mapDict[key] = idx
    print(mapDict)
    df[keyName] = df[keyName].map(mapDict)
    return df

In [5]:

def processData(df, isTrain = True):
    if isTrain:
        label = "Transported"
        df[label] = df[label].astype(int)
        df = df.drop(["PassengerId"], axis=1)
        
    df["UnkownCryoSleep"] = df["CryoSleep"].isnull()
    df[['VIP', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',"CryoSleep"]] = df[['VIP', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',"CryoSleep"]].fillna(value=0)
    df["Age"] = df["Age"].fillna(value=df["Age"].mean())
    df['VIP'] = df['VIP'].astype(int)
    df['CryoSleep'] = df['CryoSleep'].astype(int)
    df['UnkownCryoSleep'] = df['UnkownCryoSleep'].astype(int)
    df[["Deck", "Cabin_num", "Side"]] = df["Cabin"].str.split("/", expand=True)
    df["Deck"] = df["Deck"].fillna(value="UnknownDeck")
    df["Destination"] = df["Destination"].fillna(value="UnknownDestination")
    df["HomePlanet"] = df["HomePlanet"].fillna(value="UnknownHomePlanet")
    df["Side"] = df["Side"].fillna(value="UnknownSide")
    df = mapValue(df,"HomePlanet")
    df = mapValue(df,"Destination")
    df = mapValue(df,"Deck")
    df = mapValue(df,"Side")
    df = df.drop(["Cabin","Name","FoodCourt","RoomService",'ShoppingMall','Spa', 'VRDeck',"Cabin_num"], axis=1)
    return df



In [6]:
train_df = processData(train_df, isTrain=True)
test_df = processData(test_df, isTrain=False)
train_df.to_csv("./dataset/train_preprocessed.csv", index=False)
test_df.to_csv("./dataset/test_preprocessed.csv", index=False)

{'Earth': 0, 'Europa': 1, 'Mars': 2, 'UnknownHomePlanet': 3}
{'TRAPPIST-1e': 0, '55 Cancri e': 1, 'PSO J318.5-22': 2, 'UnknownDestination': 3}
{'F': 0, 'G': 1, 'E': 2, 'B': 3, 'C': 4, 'D': 5, 'A': 6, 'UnknownDeck': 7, 'T': 8}
{'S': 0, 'P': 1, 'UnknownSide': 2}
{'Earth': 0, 'Europa': 1, 'Mars': 2, 'UnknownHomePlanet': 3}
{'TRAPPIST-1e': 0, '55 Cancri e': 1, 'PSO J318.5-22': 2, 'UnknownDestination': 3}
{'F': 0, 'G': 1, 'E': 2, 'B': 3, 'C': 4, 'D': 5, 'UnknownDeck': 6, 'A': 7, 'T': 8}
{'S': 0, 'P': 1, 'UnknownSide': 2}
